In [1]:
import pandas as pd, numpy as np, warnings, os
#import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import RFE
import pickle

In [2]:
pwd

'/Users/adityabhatkal/Desktop/Changes addy-bhatkal-portfolio _ being made'

In [3]:
warnings.filterwarnings('ignore')

In [4]:
pwd

'/Users/adityabhatkal/Desktop/Changes addy-bhatkal-portfolio _ being made'

In [5]:
#os.chdir('C:\\Users\\ABC\\6. Datasets')

In [6]:
df = pd.read_csv('superstore_data_Classifier - 82% accuracy.csv')
df.columns

Index(['Id', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'Complain', 'Response'],
      dtype='object')

In [7]:
df.shape

(2240, 22)

In [8]:
df.columns
df.drop(['Id', 'Year_Birth', 'Dt_Customer'], inplace=True, axis = 1)

In [9]:
df['Education'] = df['Education'].map({
    'Basic':0,
    '2n Cycle':1,
    'Graduation':2,
    'Master':3,
    'PhD':4   
})

In [10]:
#Checking the below proportions of 1vs0 for double vs single,  singles opt in for schemes a lot more.
#SO giving them a 2
df['Marital_Status'].value_counts()

df['Marital_Status'] = df['Marital_Status'].map({
    'Married':1,
    'Together':1,
    'Single':2,
    'Widow':2,
    'Divorced':2,
    'YOLO':2,
    'Alone':2,
    'Absurd':2
})

In [11]:
listy = []
for i in df['Marital_Status'].values:
    if i not in listy:
        listy.append(i)
listy

[np.int64(2), np.int64(1)]

In [12]:
df[df['Marital_Status']=='Divorced']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [13]:
df[df['Marital_Status']=='Together']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [14]:
df[df['Marital_Status']=='Single']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [15]:
df[df['Marital_Status']=='Widow']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [16]:
df[df['Marital_Status']=='Married']['Response'].value_counts()


Series([], Name: count, dtype: int64)

In [35]:
df.describe()

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
count,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000
mean,2.460268,1.355357,52247.248661,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,27.062946,44.021875,2.325000,4.084821,2.662054,5.790179,5.316518,0.009375,0.149107
std,1.004337,0.478728,25037.797168,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,41.280498,52.167439,1.932238,2.778714,2.923101,3.250958,2.426645,0.096391,0.356274
min,0.000000,1.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,35538.750000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,0.000000,3.000000,3.000000,0.000000,0.000000
50%,2.000000,1.000000,51741.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,8.000000,24.000000,2.000000,4.000000,2.000000,5.000000,6.000000,0.000000,0.000000
75%,3.000000,2.000000,68289.750000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,33.000000,56.000000,3.000000,6.000000,4.000000,8.000000,7.000000,0.000000,0.000000
max,4.000000,2.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,362.000000,15.000000,27.000000,28.000000,13.000000,20.000000,1.000000,1.000000


In [ ]:

Average billing on fruits
(range $0 - $200)
 
Num Web Purchases
(range 0 - 27)

Num Catalog Purchases
(range 0 - 28)

Num Store Purchases
(range 0 - 13)

Num Web Visits Month
(range 0 - 20)

No: 1, Yes: 2

In [18]:
df['Income'].fillna(52247.251354, inplace=True)

In [19]:
#People who bought into the product v/s thse who didnt complain!
df[df['Response']==1]['Complain'].value_counts()

Complain
0    331
1      3
Name: count, dtype: int64

In [20]:
df.head(2)

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,2,2,84835.0,0,0,0,189,104,379,111,189,218,1,4,4,6,1,0,1
1,2,2,57091.0,0,0,0,464,5,64,7,0,37,1,7,3,7,5,0,1


In [21]:
x = df.iloc[:,[0, 1, 2, 3, 4,7,  13, 14, 15, 16, 17]]
y = df.iloc[:,-1]

In [22]:
df.isnull().sum()

Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
Complain               0
Response               0
dtype: int64

In [23]:
for i in df.columns:
    df[i] = df[i].astype(int)

In [24]:
for i in df.iloc[1,:]:
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [25]:
#Logistic Regresion for 

x_train_lr, x_test_lr, y_train_lr, y_test_lr = train_test_split(x,y,test_size=0.2, random_state=i)
lg = LogisticRegression(max_iter=1000)
model_lr = lg.fit(x_train_lr, y_train_lr)
y_predict_lr = model_lr.predict(x_test_lr)
acc_lr = accuracy_score(y_predict_lr, y_test_lr)
cv = cross_val_score(model_lr, x, y, cv=5, scoring='accuracy')
if acc_lr>0.59:
    print(i, acc_lr)
    print(cv) 

1 0.8325892857142857
[0.83705357 0.85491071 0.85714286 0.84375    0.84151786]


In [26]:
#Features to be selected
rfe = RFE(model_lr, n_features_to_select=9)
fitter = rfe.fit(x,y) 
feat = []
[feat.append(i) for i,j in enumerate(fitter.support_) if j == True]
feat

[0, 1, 3, 4, 6, 7, 8, 9, 10]

In [27]:
x_train_svc, x_test_svc, y_train_svc, y_test_svc = train_test_split(x,y,test_size = 0.2,
                                                                    random_state = 5)
svc = SVC(C=0.1, probability=True)
model_svc = svc.fit(x_train_svc, y_train_svc)
y_predict_svc = model_svc.predict(x_test_svc)
acc_svc = accuracy_score(y_test_svc, y_predict_svc)
cv = cross_val_score(svc, x,y,scoring='accuracy', cv=StratifiedKFold(n_splits=5))
if acc_svc > 0.8:
    print(cv, acc_svc)

[0.85044643 0.85044643 0.85044643 0.85044643 0.85267857] 0.8370535714285714


In [28]:
[i for i in x.iloc[4, :]]

[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]

In [29]:
print(model_svc.predict_proba([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0][1])
print(model_svc.predict([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0])

0.11929449522327568
0


In [30]:
with open('customerbuyin.sav', 'wb') as f1:
    pickle.dump(model_svc, f1)

In [31]:
with open('customerbuyin.sav', 'rb') as f2:
    cby_loaded_model = pickle.load(f2)

In [32]:
cby_loaded_model.predict_proba([[2.0, 2.0, 21474.0, 1.0, 0.0, 16.0, 3.0, 1.0, 2.0, 7.0, 0.0]])[0][1]

np.float64(0.11929449522327568)

In [33]:
x.columns

Index(['Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
       'MntFruits', 'NumWebPurchases', 'NumCatalogPurchases',
       'NumStorePurchases', 'NumWebVisitsMonth', 'Complain'],
      dtype='object')

In [34]:
model_svc.coef_()

AttributeError: coef_ is only available when using a linear kernel